In [46]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# import tensorflow as tf
# from tensorflow import keras

In [47]:
df = pd.read_csv("C:\\Users\\earn2\\Desktop\\Capstone Project\\Twitter notebook and model\\Twitter_Data.csv")
df.head()

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0


In [48]:
df.shape

(162980, 2)

In [49]:
df.isnull().sum()

clean_text    4
category      7
dtype: int64

In [50]:
df = df.dropna()
df.isnull().sum()

clean_text    0
category      0
dtype: int64

In [51]:
df['clean_text'].duplicated().sum()

0

In [52]:
df_cleaned = df

In [53]:
label = pd.get_dummies(df_cleaned.category)
label.columns = ["negative", "neutral", "positive"]
label.head()

,negative,neutral,positive
0,1,0,0
1,0,1,0
2,0,0,1
3,0,0,1
4,0,0,1


In [54]:
text_only = df_cleaned.drop(columns = "category")
text_only.head()

,clean_text
0,when modi promised “minimum government maximum...
1,talk all the nonsense and continue all the dra...
2,what did just say vote for modi welcome bjp t...
3,asking his supporters prefix chowkidar their n...
4,answer who among these the most powerful world...


## Tokenization

In [55]:
import tensorflow as tf
from tensorflow import keras

In [56]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

In [57]:
tokenizer = Tokenizer(num_words = 8150, lower = True, split = " ", oov_token = "~")
tokenizer.fit_on_texts(text_only["clean_text"])

In [58]:
word_ind = tokenizer.word_index
len(word_ind)

113679

In [59]:
list(word_ind.keys())[:10]

['~', 'modi', 'the', 'and', 'for', 'you', 'not', 'will', 'this', 'india']

In [60]:
text_only["clean_text"] = tokenizer.texts_to_sequences(text_only["clean_text"])

In [61]:
text_only.head()
#match word with position no. in index_word

,clean_text
0,"[43, 2, 308, 1, 67, 1727, 1, 1120, 41, 2379, 3..."
1,"[287, 17, 3, 1105, 4, 719, 17, 3, 810, 8, 39, ..."
2,"[25, 54, 42, 100, 39, 5, 2, 1004, 22, 509, 6, ..."
3,"[384, 20, 497, 4731, 114, 44, 1222, 2, 54, 127..."
4,"[391, 23, 814, 80, 3, 174, 927, 128, 137, 125,..."


In [62]:
len(text_only.clean_text[0]), len(text_only.clean_text[1])
#len of each tweets are not the same

(33, 13)

In [63]:
tweets = pad_sequences(text_only["clean_text"])
#make sequence have same shape

In [64]:
tweets[0].shape, tweets[1].shape

((52,), (52,))

In [65]:
#tweets[0][4:]

In [66]:
#tokenizer.fit_on_texts(['i hate you so much because you are ugly'])

In [67]:
#text_input = tokenizer.texts_to_sequences(['i hate you so much because you are ugly'])

In [68]:
#tweets_2 = pad_sequences(text_input)

## Split Data

In [69]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tweets, label, test_size = 0.15)

In [70]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.2)

In [71]:
print(f"""
Training set: tweets = {X_train.shape}, labels = {y_train.shape},
Validation set: tweets = {X_valid.shape}, labels = {y_valid.shape},
Test set: tweets = {X_test.shape}, labels = {y_test.shape}
""")


Training set: tweets = (110818, 52), labels = (110818, 3),
Validation set: tweets = (27705, 52), labels = (27705, 3),
Test set: tweets = (24446, 52), labels = (24446, 3)



## Train Model

In [72]:
model = keras.models.Sequential([
    keras.layers.Embedding(input_dim = 8150, output_dim = 32),
    keras.layers.GRU(128),
    keras.layers.Dense(128, activation = "leaky_relu", kernel_initializer = "he_normal", kernel_regularizer = "l1"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(3, activation = "softmax", kernel_initializer = "glorot_normal")
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 32)          260800    
                                                                 
 gru_1 (GRU)                 (None, 128)               62208     
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 3)                 387       
                                                                 
Total params: 339,907
Trainable params: 339,907
Non-trainable params: 0
_________________________________________________________________


In [73]:
model.compile(optimizer = 'adam',loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [74]:
history = model.fit(
    X_train, y_train, 
    epochs = 500, validation_data = (X_valid, y_valid),
    callbacks = [keras.callbacks.EarlyStopping(patience = 3, restore_best_weights = True)]
)

Epoch 1/500
3464/3464 [==============================] - 104s 30ms/step - loss: 0.9465 - accuracy: 0.8432 - val_loss: 0.3431 - val_accuracy: 0.9359
Epoch 2/500
3464/3464 [==============================] - 98s 28ms/step - loss: 0.3131 - accuracy: 0.9525 - val_loss: 0.2784 - val_accuracy: 0.9607
Epoch 3/500
3464/3464 [==============================] - 111s 32ms/step - loss: 0.2676 - accuracy: 0.9645 - val_loss: 0.2694 - val_accuracy: 0.9612
Epoch 4/500
3464/3464 [==============================] - 143s 41ms/step - loss: 0.2456 - accuracy: 0.9701 - val_loss: 0.2449 - val_accuracy: 0.9668
Epoch 5/500
3464/3464 [==============================] - 108s 31ms/step - loss: 0.2293 - accuracy: 0.9739 - val_loss: 0.2349 - val_accuracy: 0.9645
Epoch 6/500
3464/3464 [==============================] - 118s 34ms/step - loss: 0.2164 - accuracy: 0.9767 - val_loss: 0.3357 - val_accuracy: 0.9576
Epoch 7/500
3464/3464 [==============================] - 117s 34ms/step - loss: 0.2092 - accuracy: 0.9787 - val_l

In [75]:
model.evaluate(X_test, y_test)

764/764 [==============================] - 12s 15ms/step - loss: 0.2339 - accuracy: 0.9648


[0.23392677307128906, 0.9647794961929321]

In [76]:
from tensorflow.keras import Model
from keras.models import load_model

In [77]:
# model.save("C:\\Users\\earn2\\Desktop\\Capstone Project\\Model\\Twit_model")

In [78]:
new_model = tf.keras.models.load_model("C:\\Users\\earn2\\Desktop\\Capstone Project\\Twitter notebook and model\\Model\\Twit_model")

In [79]:
new_model.predict(X_valid)

866/866 [==============================] - 12s 13ms/step


array([[1.8179407e-03, 8.9995883e-04, 9.9728203e-01],
       [9.9070823e-01, 3.6042314e-03, 5.6874878e-03],
       [9.7680718e-01, 6.9138291e-03, 1.6279040e-02],
       ...,
       [2.0716127e-03, 9.9432236e-04, 9.9693406e-01],
       [5.0744847e-03, 2.1577606e-03, 9.9276781e-01],
       [2.1553237e-02, 5.2943402e-03, 9.7315246e-01]], dtype=float32)

In [80]:
new_model.evaluate(X_valid, y_valid)

866/866 [==============================] - 13s 15ms/step - loss: 0.2618 - accuracy: 0.9639


[0.26180246472358704, 0.9638693332672119]

In [81]:
def sentiment_analyzer(text):
    tokenizer.fit_on_texts([text])
    text_input = tokenizer.texts_to_sequences([text])
    tweets = pad_sequences(text_input)
    
    prediction = new_model.predict(tweets)
    
    a = ['negative', 'neutral', 'positive']
    return a[np.argmax(prediction)], prediction.max()

In [82]:
result = sentiment_analyzer('''
Swapping batteries is stupid
''')
result

1/1 [==============================] - 0s 312ms/step


('negative', 0.9621541)